In [1]:
import pandas as pd
import time
from Functions import xbrl_dict,filings_collector
import datetime as dt
import requests
import xml.etree.ElementTree as ET
from pymongo import MongoClient

In [2]:
ticker = input('ticker?').lower()

ticker?ba


In [ ]:
# Access page
all_filings_df = filings_collector(ticker)
all_filings_df.to_csv(f'../data/all_{ticker}_filings.csv', index=False)
all_filings_path = f'../data/all_{ticker}_filings.csv'

In [3]:
all_filings_path = f'../data/all_{ticker}_filings.csv'
xml_filings_df = pd.read_csv(all_filings_path)
xml_filings_df = xml_filings_df.dropna(subset=['xbrl_link'])
xml_filings_df = xml_filings_df.reset_index(drop=True)
xml_filings_df.head(100)

,date,doc_link,filings,xbrl_link
0,2019-02-08,https://www.sec.gov/Archives/edgar/data/12927/...,10-K,https://www.sec.gov/Archives/edgar/data/12927/...
1,2018-10-24,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
2,2018-07-25,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
3,2018-04-25,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
4,2018-02-12,https://www.sec.gov/Archives/edgar/data/12927/...,10-K,https://www.sec.gov/Archives/edgar/data/12927/...
5,2017-10-25,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
6,2017-07-26,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
7,2017-04-26,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...
8,2017-02-08,https://www.sec.gov/Archives/edgar/data/12927/...,10-K,https://www.sec.gov/Archives/edgar/data/12927/...
9,2016-10-26,https://www.sec.gov/Archives/edgar/data/12927/...,10-Q,https://www.sec.gov/Archives/edgar/data/12927/...


In [39]:
# lxml_ten_df = xml_filings_df.loc[xml_filings_df['filings'] == '10-Q']
filings_list = []
for i in range(0,len(xml_filings_df)):
    lxml_link = xml_filings_df['xbrl_link'][i]

    lxml_dict = xbrl_dict(lxml_link)
    
    filings_list.append(lxml_dict)


In [40]:
all_data_df = pd.DataFrame(filings_list)
columns = all_data_df.columns.tolist()

In [41]:
establising_headers = ['company_name', 'CIK','doc_date',
              'doc_year','doc_period','common_shares_outstanding',
              'amendment_flag']
for header in establising_headers:
    index = columns.index(header)
    columns.pop(index)

ordered_columns = establising_headers+columns


In [42]:
all_data_df = all_data_df[ordered_columns]
all_data_df.tail(20)

,company_name,CIK,doc_date,doc_year,doc_period,common_shares_outstanding,amendment_flag,AccrualForEnvironmentalLossContingenciesIncreaseDecreaseForRevisionInEstimates,AccrualForEnvironmentalLossContingenciesPayments,AccrualForEnvironmentalLossContingenciesPayments1,...,UnrecognizedTaxBenefitsDecreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsDecreasesResultingFromSettlementsWithTaxingAuthorities,UnrecognizedTaxBenefitsIncreasesResultingFromCurrentPeriodTaxPositions,UnrecognizedTaxBenefitsIncreasesResultingFromPriorPeriodTaxPositions,UnrecognizedTaxBenefitsReductionsResultingFromLapseOfApplicableStatuteOfLimitations,WeightedAverageNumberDilutedSharesOutstandingAdjustment,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfShareOutstandingBasicAndDiluted,WeightedAverageNumberOfSharesOutstandingBasic,exceptions
19,BOEING CO,0000012927,2014-03-31,2014,Q1,729224508.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
20,BOEING CO,0000012927,2013-12-31,2013,FY,743404506.0,false,-59000000.0,120000000.0,NaN,...,125000000.0,NaN,202000000.0,10000000.0,NaN,8700000.0,767600000.0,760800000.0,758900000.0,107
21,BOEING CO,0000012927,2013-09-30,2013,Q3,751453447.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
22,BOEING CO,0000012927,2013-06-30,2013,Q2,754444329.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
23,BOEING CO,0000012927,2013-03-31,2013,Q1,758681225.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
24,BOEING CO,0000012927,2012-12-31,2012,FY,756165531.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
25,BOEING CO,0000012927,2012-09-30,2012,Q3,754076883.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
26,BOEING CO,0000012927,2012-06-30,2012,Q2,751797604.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
27,BOEING CO,0000012927,2012-03-31,2012,Q1,749049454.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
28,BOEING CO,0000012927,2011-12-31,2011,FY,745720763.0,false,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
all_years = []
for year in list(all_data_df['doc_year']):
    if year not in all_years:
         all_years.append(year)
counter = 1            
for index, row in all_data_df.iterrows():
#     year = row['doc_year']
#     period = row['doc_period']
    counter += 1
    print(row)
    print(counter)
    
    
    

In [43]:
all_data_df.to_csv(f'../data/{ticker}_financials.csv', index=False)

In [ ]:
# Mongo DB connection
client = MongoClient('mongodb://localhost:27017')
db = client.Company_Info
collection = db.SEC_filings

for filing in filings_list:
    collection.insert(filing)


In [ ]:
# uris = []
# all_names = []
# for i in range(0,len(xml_filings_df)):
#     lxml_link = xml_filings_df['xbrl_link'][i]
#     xbrl_resp = requests.get(lxml_link)
#     time.sleep(.1)
#     xbrl_str = xbrl_resp.text
#     root = ET.fromstring(xbrl_str)

#     #     defining namespace prefixes
#     uri_list = []
#     names = []
#     for child in root:
#         uri_pos = child.tag.index('}')
#         uri = ''.join(child.tag[1:uri_pos])
#         name = ''.join(child.tag[int(uri_pos+1):len(child.tag)])
        
#         if uri not in uri_list:
#             uri_list.append(uri)
        
#         if uri[0:18] == 'http://xbrl.us/dei':
#             names.append(f'{name}: {child.text}')
#     uris.append(uri_list)
#     all_names.append(names)

In [45]:
lxml_link = xml_filings_df['doc_link'][21]
print(lxml_link)

https://www.sec.gov/Archives/edgar/data/12927/000001292713000041/0000012927-13-000041-index.htm
